In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max
from PIL import Image
from skimage.morphology import remove_small_objects


from skimage.morphology import skeletonize, thin
from skimage.util import invert
from skimage import measure
from skimage.morphology import erosion, dilation, opening, closing, white_tophat
from skimage.morphology import black_tophat, skeletonize, convex_hull_image
from skimage.morphology import disk
from skimage import feature
from skimage.filters import roberts, sobel, scharr, prewitt
import seaborn as sns
sns.reset_orig()

from skimage import restoration

from skimage import exposure
from skimage.filters import gaussian

from scipy import ndimage

from time import time

from skimage.color import rgb2gray
from scipy import ndimage as ndi

from skimage.filters.rank import entropy
import scipy


import networkx as nx
from scipy import interpolate

from skimage.segmentation import random_walker
from skimage.data import binary_blobs
from skimage.exposure import rescale_intensity
import glob

from skimage.morphology import disk
from skimage import io

from skimage.color import rgb2hsv
import cv2

from skimage.external import tifffile

import pandas as pd

from sklearn.cluster import KMeans

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

import threading

/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:855: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
/Users/spencerlab/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:846: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 2.2 and will be removed in 3.1.
  "2.2", name=key, obj_type="rcparam", addendum=addendum)


In [2]:
def get_occurances(arr):
    
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    points = np.where(arr)
    points = np.asarray(points)
    
    pt_labels = markers[points[0], points[1], points[2]]

    label_occurances = (np.bincount(pt_labels, weights=np.ones((points.shape[1]))) )[1:]
    
    
    return label_occurances

In [3]:
def color_leave(arr, h_min, h_max, s_min, s_max, v_min, v_max):
    arr2= arr*0
    for i in range(len(arr)):
        arr2[i] =  cv2.cvtColor(arr[i],cv2.COLOR_RGB2HSV)
        
    
    if h_max<h_min:
        h_max_2 = h_max.copy()
        h_max = h_min
        h_min = h_max_2
    
    
    m = (arr2[:, :,:,0]>=h_min)*(arr2[:, :,:,0]<=h_max)*(arr2[:,:,:,1]>=s_min)*(arr2[:,:,:,1]<=s_max)*(arr2[:,:,:,2]>=v_min)*(arr2[:,:,:,2]<=v_max)
    
    return m
    

In [4]:
def get_centroids(arr):
    
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    points = np.where(arr)
    points = np.asarray(points)
    
    pt_labels = markers[points[0], points[1], points[2]]
    
    Z_mass = (np.bincount(pt_labels, weights=points[0]))[1:]

    Y_mass = (np.bincount(pt_labels, weights=points[1]))[1:]

    X_mass = (np.bincount(pt_labels, weights=points[2]))[1:]

    label_occurances = (np.bincount(pt_labels, weights=np.ones((points.shape[1]))) )[1:]
    
    
    Z_centroid = Z_mass/label_occurances
    Y_centroid = Y_mass/label_occurances
    X_centroid = X_mass/label_occurances
    
    return Z_centroid, Y_centroid, X_centroid, label_occurances

In [5]:
def size_filter(arr, thresh, occurances):
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    points = np.where(arr)
    points = np.asarray(points)
    
    markers[points[0], points[1], points[2]] = occurances*(occurances>=thresh)
    
    

In [6]:
def pair_labels_W_vals(arr, vals):
    arr= arr!=0
    markers, nummarkers = ndi.label(arr) 
    
    labels = np.arange(nummarkers)+1
    
    
    points = np.where(arr)
    points = np.asarray(points)
    
    pt_labels = markers[points[0], points[1], points[2]]
    
    dict_1 = dict(zip(labels, vals))
    
    new_labels = np.vectorize(dict_1.get)(pt_labels)
    
    markers[points[0], points[1], points[2]] = new_labels
    
    return markers

In [28]:
#stack = io.imread('/Users/spencerlab/Desktop/final_files/Day_2/#1/715_2d_RGB_copy.tif')

stack = io.imread('/Users/spencerlab/Desktop/final_files/Day_5/Stack_day_5.tif')

In [29]:
#PARAMETERS
h_min = 52
s_min = 141
v_min = 145

h_max = 116
s_max = 255
v_max = 255


MINIMUM_SIZE = 12


In [30]:
filtered_stack = color_leave(stack, h_min, h_max, s_min, s_max, v_min, v_max)

In [31]:
occ = get_occurances(filtered_stack)

In [32]:
sizemap_stack = pair_labels_W_vals(filtered_stack, occ)

In [33]:
Z_centroid, Y_centroid, X_centroid, label_occurances = get_centroids(sizemap_stack>=MINIMUM_SIZE)

In [34]:
Z_centroid.shape[0]

70404

In [35]:
data = np.zeros((4, Z_centroid.shape[0]), dtype='uint8')
data[0] = Z_centroid
data[1] = Y_centroid
data[2] = X_centroid
data[3] = label_occurances
data = np.transpose(data)

In [36]:
df = pd.DataFrame(data=data, columns=['Z', 'Y', 'X', 'Size-voxels'])

In [37]:
df

,Z,Y,X,Size-voxels
0,17,169,19,215
1,3,4,199,17
2,0,5,116,22
3,1,13,200,35
4,1,20,187,48
...,...,...,...,...
70399,236,30,203,28
70400,237,14,213,13
70401,239,231,96,13
70402,238,19,129,45


In [38]:
df.to_csv('/Users/spencerlab/Downloads/Day_2-1.csv')